In [7]:
import pandas as pd

In [44]:
df = pd.read_pickle('data.pkl')
pd.set_option('display.max_rows', 200)

In [30]:
df_base = df[['text','start_char','end_char']].copy().drop_duplicates().reset_index()
for nlp in df['nlp'].unique():
    nlp_type_column = f'{nlp}_type'
    df_base = pd.merge(left=df_base, right=df.query(f'nlp == "{nlp}"'), how='left', on=['text','start_char','end_char'])
    df_base.drop('nlp', axis=1, inplace=True)
    df_base.rename(columns={'type':nlp_type_column}, inplace=True)

In [47]:
df_base.head(200)

,index,text,start_char,end_char,Stanza-Default_type,Spacy-Default_type,FastPDN_type
0,0,28 grudnia 2023 r.,323,341,date,date,NaN
1,1,27 grudnia 2023 r.,368,386,date,date,NaN
2,2,Spółka,460,466,orgName,orgName,NaN
3,3,DFG,571,574,orgName,orgName,NaN
4,4,"„WFS""",762,767,orgName,NaN,NaN
5,5,Niemiec,857,864,placeName,placeName,nam_loc_gpe_country
6,6,Niemczech,940,949,placeName,placeName,nam_loc_gpe_country
7,7,"„Grupa""",1023,1030,orgName,NaN,NaN
8,8,Rzeczypospolitej Polskiej,1216,1241,placeName,placeName,nam_loc_gpe_country
9,9,Polsce,1293,1299,placeName,placeName,nam_loc_gpe_country
